# GenAI and DS Apps — Fall 2025
## Homework 3 — Open‑source LLM exercises (Ollama → Mistral)

**Name:** Esteban Velasquez

**Student ID:** 114502904  



## 1) Installation & start Ollama server



In [1]:
%%bash
set -e
# install ollama if not installed
if ! command -v ollama >/dev/null 2>&1; then
  curl -fsSL https://ollama.com/install.sh | sh
fi

# restart server
pkill -f "ollama serve" || true
nohup ollama serve > /content/ollama.log 2>&1 &

sleep 5
ollama --version
echo "✅ Ollama server started"


ollama version is 0.11.11
✅ Ollama server started


>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


## 2) Choose Model



In [2]:
MODEL_NAME = "mistral"   # you can switch to "phi3" or "llama3.2:1b-instruct" if mistral is too big
print("Using model:", MODEL_NAME)

Using model: mistral


## 3) Pull Model



In [3]:
import subprocess

print("Pulling model:", MODEL_NAME)
completed = subprocess.run(["ollama", "pull", MODEL_NAME], capture_output=True, text=True)
print(completed.stdout)


Pulling model: mistral



## 4) Prompt

In [4]:
HW2_PROMPT = """
Explain Bayesian analysis and Bayes’ rule in a step-by-step fashion. Show us an example in medical diagnostics (both in Mathematic and in Python code)
""".strip()

## 5) Result

In [5]:
import requests, json

def ollama_generate(model, prompt):
    r = requests.post(
        "http://localhost:11434/api/generate",
        json={"model": model, "prompt": prompt, "stream": False}
    )
    r.raise_for_status()
    return r.json()["response"]

mistral_output = ollama_generate(MODEL_NAME, HW2_PROMPT)
print(mistral_output)


 Bayesian analysis is a statistical method that uses Bayes' theorem to update the probability for a hypothesis as more evidence or information becomes available. This approach is based on the principle of conditional probability and is widely used in various fields, including machine learning, artificial intelligence, and decision-making under uncertainty.

Bayes' rule is the mathematical formula that forms the basis of Bayesian analysis. It's named after Thomas Bayes, an 18th-century statistician. The rule states that:

P(A|B) = P(B|A) * P(A) / P(B)

Where:
1. P(A|B) is the probability of event A occurring given that event B has occurred (the posterior probability).
2. P(B|A) is the probability of event B occurring given that event A has occurred (the likelihood).
3. P(A) is the prior probability of event A occurring before any evidence is considered.
4. P(B) is the probability of event B occurring (marginal likelihood or evidence).

Now, let's consider an example in medical diagnosti

## 6) Original **HW2 prompt** (Bayes' rule)
Prompt: Explain Bayesian analysis and Bayes’ rule in a step-by-step fashion. Show us an example in medical diagnostics (both in Mathematic and in Python code)

Result:


Bayesian Analysis:

Bayesian analysis is a statistical methos that uses Baye's theorem to update beliefes about unknown parameters when new data surges. It treats the parameters as random variables and sets them in a probability distribution to represent their uncertainty.

1) Big idea:

After seeing the new data, you update what you believed before by how compatible the data are with each possibility, and then normalize by the overall chance of seeing the data like this.

2) Formula:

P(0|data) = (P(data|0) * P(0)) / P(data)

Where:
- P(0): prior belief about parameter
- P(data∣0): likelihood
- P(data)=∫P(data∣0) P(0) d0 evidence normalized
- P(0∣data): updated belief

3) Formula in text:

Posterior = Likelihood × Prior ÷ Evidence.

4) Mathematical Example:

- Prevalence = P(Disease) = 0.01
- Sentitivity = P(+|Disease) = 0.9
- Specificity = P(-|Disease) = 0.05

Step 1:

P(+) = P(+|Disease) * P(Disease) + P(+|Disease) * P(-Disease)

P(+) = 0.9 * 0.01 + 0.05 * 0.99 = 0.0585

Step 2:

P(0|Disease) = (P(Disease|0) * P(0)) / P(Disease)

P(Disease|+) = 0.9 * 0.01 / 0.0585 = 0.154


In [6]:
# Parameters
prevalence = 0.01        # P(D)
sensitivity = 0.90       # P(+|D)
specificity = 0.95       # P(-|¬D)
false_positive_rate = 1 - specificity

# Step 1: Evidence (probability of a positive test)
p_positive = sensitivity * prevalence + false_positive_rate * (1 - prevalence)

# Step 2: Posterior (probability of disease given positive test)
posterior = (sensitivity * prevalence) / p_positive

print(f"Probability of disease given positive test: {posterior:.3f}")

Probability of disease given positive test: 0.154


## 7) Conclusion

When comparing the two answers, I noticed some clear differences in style and depth. Mistral gave a very structured and textbook-like explanation: it clearly defined Bayes’ rule, walked through a medical test example with correct numbers, and even included Python code that reproduced the results. ChatGPT, on the other hand, focused more on intuition, using simpler language and different ways of writing the formula, which made it easier to grasp conceptually but also introduced some notational errors and incomplete calculations. I also think the size of the models played a role here—Mistral, being a smaller open-source model, tends to rely on rigid structures and formulaic outputs, while ChatGPT, with far more parameters, can provide more flexible and nuanced explanations even if it occasionally slips in precision. Personally, I found ChatGPT’s version clearer for building intuition, but Mistral’s response was more precise and practical thanks to its step-by-step math and code.